In [1]:
import os

In [2]:
# this makes the parent file the current working directory
# NOTE - Restart whole notebook otherwise the wrong working directory will be set
os.chdir("../")

print(os.getcwd())

/Users/yyh/Documents/selfstudy/E2E-Kidney-Disease-MLOps-Project


In [ ]:
import os
import zipfile
import gdown
import torch

from dataclasses import dataclass
from pathlib import Path
from torchvision.models import vgg16

from KidneyDiseasePrediction.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH
from KidneyDiseasePrediction.utils.common import read_yaml, create_directories, get_size


@dataclass(frozen=True)
class BaseModelConfig:
    root_dir: Path
    base_model_path: Path
    modified_base_model_path: Path
    params_learning_rate: float
    params_weights: str
    params_classes: int
    params_freeze_ratio: float


class ConfigManager:
    def __init__(self, config_file_path=CONFIG_FILE_PATH, params_file_path=PARAMS_FILE_PATH):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)

        create_directories([self.config.artifacts_root])

    def get_base_model_config(self) -> BaseModelConfig:

        config = self.config.base_model

        create_directories([config.root_dir])

        base_model_config = BaseModelConfig(
            root_dir=Path(config.root_dir),
            base_model_path=Path(config.base_model_path),
            modified_base_model_path=Path(config.modified_base_model_path),
            params_learning_rate=self.params.LEARNING_RATE,
            params_weights=self.params.WEIGHTS,
            params_classes=self.params.CLASSES,
            params_freeze_ratio=self.params.FREEZE_RATIO,
        )

        return base_model_config


class BaseModel:
    def __init__(self, config: BaseModelConfig):
        self.config = config

    def get_base_model(self):
        self.model = vgg16(weights=self.config.params_weights)
        self.save_model(self.config.base_model_path, self.model)

    def modify_base_model(self):
        layers = list(self.model.features.children())
        total_layers = len(layers)
        freeze_until = int(total_layers * self.config.params_freeze_ratio)

        for i, layer in enumerate(layers):
            if i < freeze_until:
                for param in layer.parameters():
                    param.requires_grad = False

        self.model.classifier[6] = torch.nn.Linear(in_features=4096, out_features=self.config.params_classes)

        self.save_model(self.config.modified_base_model_path, self.model)

    @staticmethod
    def save_model(path: Path, model: torch.nn.Module):
        torch.save(model, path)

In [12]:
try:
    config_manager = ConfigManager()
    base_model_config = config_manager.get_base_model_config()
    base_model = BaseModel(base_model_config)
    base_model.get_base_model()
    base_model.modify_base_model()

except Exception as e:
    raise e

2025-07-10 13:48:47,698 - INFO - common - YAML file config/config.yaml loaded successfully.
2025-07-10 13:48:47,700 - INFO - common - YAML file params.yaml loaded successfully.
2025-07-10 13:48:47,701 - INFO - common - Directory artifacts created at artifacts
2025-07-10 13:48:47,702 - INFO - common - Directory artifacts/base_model created at artifacts/base_model
